In [ ]:
%cd ..

In [ ]:
from tokenizer.tokenizer import Tokenizer
from huggingface_hub import login
from trainer import TrainerConfig, Trainer
from hf_backed_trainer import HFBackedTrainer
from model.gpt2 import GPT

In [ ]:
login()

In [ ]:
trainer_cfg_file = "config/news_trainer.yml"
model_cfg_file = "config/news_model.yml"

In [ ]:
def load_config(config_file)->TrainerConfig:
    import yaml

    with open(config_file) as f:
        try:
            doc = yaml.safe_load(f)
        except yaml.YAMLError as exc:
            print(exc)
            exit(1)
    
    return TrainerConfig(**doc)

trainer_cfg = load_config(trainer_cfg_file)

In [ ]:
trainer_cfg.repo_id, trainer_cfg.ds_repo_id

## Prepare Data

In [ ]:
tokenizer = Tokenizer.from_pretrained(trainer_cfg.repo_id)

In [ ]:
tokenizer.encode_ds_from_hub(trainer_cfg.ds_repo_id, trainer_cfg.data_dir)

## Train the model

In [ ]:
model = GPT.from_config(model_cfg_file)
model.to("cuda")

In [ ]:
trainer = HFBackedTrainer.from_config(trainer_cfg_file)
trainer.train(model)